In [5]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
%matplotlib inline
style.use('bmh')
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder, OrdinalEncoder, BinaryEncoder

from sklearn.preprocessing import FunctionTransformer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from catboost import CatBoostClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [9]:
df = pd.read_csv('bank.csv', delimiter = ';', na_values = 'unknown')
df['default'].fillna('unknown', inplace = True)
df.drop('pdays',axis = 1, inplace = True)

df['y'].replace({'yes':1, 'no':0}, inplace = True)

data_types = df.dtypes
int_cols = [df.columns[i] for i in range(df.shape[1]) if data_types[i] == 'int64']
float_cols = [df.columns[i] for i in range(df.shape[1]) if data_types[i] == 'float64']
cat_cols = [df.columns[i] for i in range(df.shape[1]) if data_types[i] == 'O']

mon_num = {'mar':1, 'apr':2, 'may':3, 'jun': 4, 'jul':5, 'aug':6, 'nov':7, 'sep':7, 'oct':8, 'nov':9, 'dec':10}
day_num = {day:i for day, i in zip(df['day_of_week'].unique(), range(1, 6))}
education_sorted = {'illiterate': 0, 'unknown': 1, 'basic.4y': 2,
                    'basic.6y': 3, 'basic.9y': 4, 'high.school': 5,
                    'professional.course': 6, 'university.degree': 7}

ord_mon = OrdinalEncoder(mapping=[{'col': 'month', 'mapping': mon_num}])
ord_day =  OrdinalEncoder(mapping=[{'col': 'day_of_week', 'mapping': day_num}])
ord_edu = OrdinalEncoder(mapping=[{'col': 'education', 'mapping': education_sorted}])

cat_steps = [('encoder', BinaryEncoder(drop_invariant = True)),
            ('imputer', KNNImputer(weights = 'distance')),
              ('scaler', StandardScaler())
             ]


transf = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

transformer = ColumnTransformer(
    [
    ('encode_scale', Pipeline(steps = cat_steps), list(set(cat_cols)-set(['day_of_week', 'month', 'y']))),
    ('month', Pipeline(steps = [('ord_mon', ord_mon) , ('scale_mon', StandardScaler())]), 'month'),
    ('day', Pipeline(steps = [('ord_day', ord_day), ('scale_day', StandardScaler())]), 'day_of_week'),
    ('scale', StandardScaler(), ['age', 'duration', *float_cols, 'previous', 'campaign'])
    ]
)
    

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df, df['y'], test_size = 0.2, random_state = 69)

X_train = transformer.fit_transform(X_train);
X_test = transformer.transform(X_test);

C:\Users\Eugene\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Eugene\anaconda3\lib\site-packages\category_encoders\basen.py:236: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_out.drop(col, 1, inplace=True)
C:\Users\Eugene\anaconda3\lib\site-packages\category_encoders\basen.py:236: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_out.drop(col, 1, inplace=True)


In [24]:
from matplotlib.pyplot import cm

def search_best(X, y, model, param_grid, param_scale='ordinary', draw=True, multi = False, scoring = 'f1', error_score='raise'):
    
    CV_model = GridSearchCV(estimator=model, 
                            param_grid=param_grid,
                            cv=3, 
                            scoring= scoring,
                            n_jobs=1, 
                            verbose=1,
                           refit = False)
    CV_model.fit(X, y)

    return CV_model

In [28]:
# Random Forest не переобучается из-за количества деревьев, поэтому варьировать его нет смысла, метрика будет улучшаться
models = [
    CatBoostClassifier(),
    AdaBoostClassifier(),
    RandomForestClassifier()
]
param_grids = [
    {
        'depth':np.arange(2, 7)},
               {'n_estimators': np.array([25, 50, 75, 100])},
    {
                'min_samples_split':[2, 4, 6, 8],
               'min_samples_leaf': np.arange(1, 5),
                'min_weight_fraction_leaf' : np.linspace(0, 0.5, 5)
               }
              ]


In [29]:
res = []
for model, param_grid in zip(models, param_grids):
    res.append(search_best(X_train, y_train, model, param_grid, scoring = 'f1', draw = False))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Learning rate set to 0.038536
0:	learn: 0.6492395	total: 162ms	remaining: 2m 42s
1:	learn: 0.6055003	total: 167ms	remaining: 1m 23s
2:	learn: 0.5625936	total: 172ms	remaining: 57.1s
3:	learn: 0.5240957	total: 177ms	remaining: 44s
4:	learn: 0.4899475	total: 183ms	remaining: 36.4s
5:	learn: 0.4698740	total: 187ms	remaining: 31s
6:	learn: 0.4487424	total: 192ms	remaining: 27.3s
7:	learn: 0.4303198	total: 198ms	remaining: 24.5s
8:	learn: 0.4150371	total: 203ms	remaining: 22.4s
9:	learn: 0.3954877	total: 209ms	remaining: 20.6s
10:	learn: 0.3814248	total: 214ms	remaining: 19.3s
11:	learn: 0.3668707	total: 219ms	remaining: 18s
12:	learn: 0.3515995	total: 224ms	remaining: 17s
13:	learn: 0.3418382	total: 230ms	remaining: 16.2s
14:	learn: 0.3327296	total: 234ms	remaining: 15.4s
15:	learn: 0.3196083	total: 239ms	remaining: 14.7s
16:	learn: 0.3133092	total: 244ms	remaining: 14.1s
17:	learn: 0.3061784	total: 249ms	remaining: 13.6s
18:	lear

172:	learn: 0.1918463	total: 1.09s	remaining: 5.23s
173:	learn: 0.1918021	total: 1.1s	remaining: 5.23s
174:	learn: 0.1917181	total: 1.11s	remaining: 5.26s
175:	learn: 0.1915471	total: 1.12s	remaining: 5.26s
176:	learn: 0.1914960	total: 1.13s	remaining: 5.26s
177:	learn: 0.1914592	total: 1.14s	remaining: 5.26s
178:	learn: 0.1914022	total: 1.14s	remaining: 5.24s
179:	learn: 0.1913556	total: 1.15s	remaining: 5.23s
180:	learn: 0.1912878	total: 1.15s	remaining: 5.22s
181:	learn: 0.1912273	total: 1.16s	remaining: 5.21s
182:	learn: 0.1912036	total: 1.17s	remaining: 5.2s
183:	learn: 0.1910003	total: 1.17s	remaining: 5.19s
184:	learn: 0.1909367	total: 1.18s	remaining: 5.18s
185:	learn: 0.1908722	total: 1.18s	remaining: 5.17s
186:	learn: 0.1908409	total: 1.19s	remaining: 5.16s
187:	learn: 0.1908035	total: 1.19s	remaining: 5.15s
188:	learn: 0.1907155	total: 1.2s	remaining: 5.14s
189:	learn: 0.1906518	total: 1.2s	remaining: 5.13s
190:	learn: 0.1906179	total: 1.21s	remaining: 5.12s
191:	learn: 0.19

347:	learn: 0.1844321	total: 2.19s	remaining: 4.1s
348:	learn: 0.1844162	total: 2.19s	remaining: 4.09s
349:	learn: 0.1843986	total: 2.2s	remaining: 4.09s
350:	learn: 0.1843772	total: 2.21s	remaining: 4.08s
351:	learn: 0.1843519	total: 2.21s	remaining: 4.08s
352:	learn: 0.1843481	total: 2.22s	remaining: 4.07s
353:	learn: 0.1843255	total: 2.23s	remaining: 4.07s
354:	learn: 0.1843044	total: 2.23s	remaining: 4.06s
355:	learn: 0.1842755	total: 2.24s	remaining: 4.05s
356:	learn: 0.1842536	total: 2.25s	remaining: 4.05s
357:	learn: 0.1842155	total: 2.25s	remaining: 4.04s
358:	learn: 0.1841745	total: 2.26s	remaining: 4.04s
359:	learn: 0.1841354	total: 2.27s	remaining: 4.03s
360:	learn: 0.1840988	total: 2.27s	remaining: 4.03s
361:	learn: 0.1840482	total: 2.28s	remaining: 4.02s
362:	learn: 0.1840336	total: 2.29s	remaining: 4.01s
363:	learn: 0.1840285	total: 2.29s	remaining: 4.01s
364:	learn: 0.1840016	total: 2.3s	remaining: 4s
365:	learn: 0.1839460	total: 2.31s	remaining: 4s
366:	learn: 0.1839249

521:	learn: 0.1807360	total: 3.48s	remaining: 3.19s
522:	learn: 0.1807224	total: 3.49s	remaining: 3.18s
523:	learn: 0.1807194	total: 3.5s	remaining: 3.18s
524:	learn: 0.1807087	total: 3.51s	remaining: 3.17s
525:	learn: 0.1806963	total: 3.51s	remaining: 3.17s
526:	learn: 0.1806875	total: 3.52s	remaining: 3.16s
527:	learn: 0.1806316	total: 3.53s	remaining: 3.15s
528:	learn: 0.1806192	total: 3.54s	remaining: 3.15s
529:	learn: 0.1806050	total: 3.55s	remaining: 3.14s
530:	learn: 0.1805899	total: 3.55s	remaining: 3.14s
531:	learn: 0.1805767	total: 3.56s	remaining: 3.13s
532:	learn: 0.1805606	total: 3.57s	remaining: 3.13s
533:	learn: 0.1805482	total: 3.58s	remaining: 3.12s
534:	learn: 0.1805327	total: 3.59s	remaining: 3.12s
535:	learn: 0.1805186	total: 3.6s	remaining: 3.11s
536:	learn: 0.1805106	total: 3.6s	remaining: 3.11s
537:	learn: 0.1804987	total: 3.61s	remaining: 3.1s
538:	learn: 0.1804726	total: 3.62s	remaining: 3.09s
539:	learn: 0.1804576	total: 3.63s	remaining: 3.09s
540:	learn: 0.18

686:	learn: 0.1775723	total: 4.8s	remaining: 2.19s
687:	learn: 0.1775499	total: 4.8s	remaining: 2.18s
688:	learn: 0.1774667	total: 4.81s	remaining: 2.17s
689:	learn: 0.1774565	total: 4.82s	remaining: 2.16s
690:	learn: 0.1774315	total: 4.82s	remaining: 2.16s
691:	learn: 0.1774207	total: 4.83s	remaining: 2.15s
692:	learn: 0.1774192	total: 4.84s	remaining: 2.14s
693:	learn: 0.1774107	total: 4.85s	remaining: 2.14s
694:	learn: 0.1773988	total: 4.85s	remaining: 2.13s
695:	learn: 0.1773209	total: 4.86s	remaining: 2.12s
696:	learn: 0.1773042	total: 4.87s	remaining: 2.12s
697:	learn: 0.1772315	total: 4.87s	remaining: 2.11s
698:	learn: 0.1772115	total: 4.88s	remaining: 2.1s
699:	learn: 0.1771900	total: 4.89s	remaining: 2.09s
700:	learn: 0.1771690	total: 4.89s	remaining: 2.09s
701:	learn: 0.1771589	total: 4.9s	remaining: 2.08s
702:	learn: 0.1771461	total: 4.91s	remaining: 2.07s
703:	learn: 0.1771386	total: 4.92s	remaining: 2.07s
704:	learn: 0.1771174	total: 4.93s	remaining: 2.06s
705:	learn: 0.17

...

188:	learn: 0.1632570	total: 2.59s	remaining: 11.1s
189:	learn: 0.1631985	total: 2.6s	remaining: 11.1s
190:	learn: 0.1631080	total: 2.62s	remaining: 11.1s
191:	learn: 0.1630195	total: 2.63s	remaining: 11.1s
192:	learn: 0.1629207	total: 2.64s	remaining: 11.1s
193:	learn: 0.1628719	total: 2.66s	remaining: 11s
194:	learn: 0.1627541	total: 2.67s	remaining: 11s
195:	learn: 0.1626688	total: 2.68s	remaining: 11s
196:	learn: 0.1625258	total: 2.69s	remaining: 11s
197:	learn: 0.1624613	total: 2.71s	remaining: 11s
198:	learn: 0.1623712	total: 2.72s	remaining: 11s
199:	learn: 0.1623103	total: 2.73s	remaining: 10.9s
200:	learn: 0.1622212	total: 2.75s	remaining: 10.9s
201:	learn: 0.1621688	total: 2.76s	remaining: 10.9s
202:	learn: 0.1620806	total: 2.77s	remaining: 10.9s
203:	learn: 0.1619406	total: 2.79s	remaining: 10.9s
204:	learn: 0.1617933	total: 2.8s	remaining: 10.9s
205:	learn: 0.1617119	total: 2.82s	remaining: 10.9s
206:	learn: 0.1616273	total: 2.83s	remaining: 10.8s
207:	learn: 0.1615180	tota

351:	learn: 0.1500754	total: 4.79s	remaining: 8.82s
352:	learn: 0.1499851	total: 4.8s	remaining: 8.81s
353:	learn: 0.1499372	total: 4.82s	remaining: 8.79s
354:	learn: 0.1498576	total: 4.83s	remaining: 8.78s
355:	learn: 0.1497949	total: 4.84s	remaining: 8.76s
356:	learn: 0.1497442	total: 4.85s	remaining: 8.74s
357:	learn: 0.1496582	total: 4.87s	remaining: 8.73s
358:	learn: 0.1496218	total: 4.88s	remaining: 8.71s
359:	learn: 0.1495736	total: 4.89s	remaining: 8.7s
360:	learn: 0.1495189	total: 4.9s	remaining: 8.68s
361:	learn: 0.1494353	total: 4.92s	remaining: 8.66s
362:	learn: 0.1493468	total: 4.93s	remaining: 8.66s
363:	learn: 0.1492707	total: 4.95s	remaining: 8.64s
364:	learn: 0.1492277	total: 4.96s	remaining: 8.63s
365:	learn: 0.1491900	total: 4.97s	remaining: 8.61s
366:	learn: 0.1491325	total: 4.98s	remaining: 8.6s
367:	learn: 0.1490376	total: 5s	remaining: 8.58s
368:	learn: 0.1489836	total: 5.01s	remaining: 8.57s
369:	learn: 0.1489418	total: 5.02s	remaining: 8.55s
370:	learn: 0.14885

524:	learn: 0.1394604	total: 7.18s	remaining: 6.49s
525:	learn: 0.1393922	total: 7.19s	remaining: 6.48s
526:	learn: 0.1393525	total: 7.2s	remaining: 6.47s
527:	learn: 0.1393193	total: 7.22s	remaining: 6.45s
528:	learn: 0.1392838	total: 7.23s	remaining: 6.43s
529:	learn: 0.1392205	total: 7.24s	remaining: 6.42s
530:	learn: 0.1391268	total: 7.25s	remaining: 6.41s
531:	learn: 0.1390705	total: 7.26s	remaining: 6.39s
532:	learn: 0.1390143	total: 7.28s	remaining: 6.38s
533:	learn: 0.1389335	total: 7.29s	remaining: 6.36s
534:	learn: 0.1388890	total: 7.3s	remaining: 6.35s
535:	learn: 0.1388420	total: 7.31s	remaining: 6.33s
536:	learn: 0.1387807	total: 7.33s	remaining: 6.32s
537:	learn: 0.1387068	total: 7.34s	remaining: 6.3s
538:	learn: 0.1386411	total: 7.35s	remaining: 6.29s
539:	learn: 0.1385801	total: 7.36s	remaining: 6.27s
540:	learn: 0.1385448	total: 7.38s	remaining: 6.26s
541:	learn: 0.1385032	total: 7.39s	remaining: 6.24s
542:	learn: 0.1384703	total: 7.4s	remaining: 6.23s
543:	learn: 0.13

690:	learn: 0.1307891	total: 9.36s	remaining: 4.19s
691:	learn: 0.1307472	total: 9.38s	remaining: 4.17s
692:	learn: 0.1307056	total: 9.39s	remaining: 4.16s
693:	learn: 0.1306452	total: 9.4s	remaining: 4.14s
694:	learn: 0.1306065	total: 9.41s	remaining: 4.13s
695:	learn: 0.1305621	total: 9.43s	remaining: 4.12s
696:	learn: 0.1305192	total: 9.44s	remaining: 4.1s
697:	learn: 0.1304762	total: 9.45s	remaining: 4.09s
698:	learn: 0.1304255	total: 9.46s	remaining: 4.07s
699:	learn: 0.1303986	total: 9.47s	remaining: 4.06s
700:	learn: 0.1303663	total: 9.49s	remaining: 4.05s
701:	learn: 0.1303277	total: 9.5s	remaining: 4.03s
702:	learn: 0.1302737	total: 9.52s	remaining: 4.02s
703:	learn: 0.1302100	total: 9.53s	remaining: 4.01s
704:	learn: 0.1301380	total: 9.54s	remaining: 3.99s
705:	learn: 0.1300818	total: 9.55s	remaining: 3.98s
706:	learn: 0.1300515	total: 9.56s	remaining: 3.96s
707:	learn: 0.1300017	total: 9.58s	remaining: 3.95s
708:	learn: 0.1299707	total: 9.59s	remaining: 3.94s
709:	learn: 0.1

857:	learn: 0.1234180	total: 11.6s	remaining: 1.92s
858:	learn: 0.1233879	total: 11.6s	remaining: 1.9s
859:	learn: 0.1233458	total: 11.6s	remaining: 1.89s
860:	learn: 0.1233176	total: 11.6s	remaining: 1.87s
861:	learn: 0.1232705	total: 11.6s	remaining: 1.86s
862:	learn: 0.1232421	total: 11.6s	remaining: 1.85s
863:	learn: 0.1231919	total: 11.6s	remaining: 1.83s
864:	learn: 0.1231613	total: 11.7s	remaining: 1.82s
865:	learn: 0.1231352	total: 11.7s	remaining: 1.8s
866:	learn: 0.1231116	total: 11.7s	remaining: 1.79s
867:	learn: 0.1230368	total: 11.7s	remaining: 1.78s
868:	learn: 0.1229879	total: 11.7s	remaining: 1.76s
869:	learn: 0.1229688	total: 11.7s	remaining: 1.75s
870:	learn: 0.1229214	total: 11.7s	remaining: 1.74s
871:	learn: 0.1229005	total: 11.7s	remaining: 1.72s
872:	learn: 0.1228633	total: 11.8s	remaining: 1.71s
873:	learn: 0.1228334	total: 11.8s	remaining: 1.7s
874:	learn: 0.1228065	total: 11.8s	remaining: 1.68s
875:	learn: 0.1227606	total: 11.8s	remaining: 1.67s
876:	learn: 0.1

In [30]:
best_params = [model.best_params_ for model in res]
best_score_test = [model.best_score_ for model in res]
best_params

[{'depth': 6},
 {'n_estimators': 100},
 {'min_samples_leaf': 2,
  'min_samples_split': 4,
  'min_weight_fraction_leaf': 0.0}]

In [36]:
cboost = CatBoostClassifier(depth = 6)
aboost = AdaBoostClassifier(n_estimators = 100)
rf = RandomForestClassifier( **{'min_samples_leaf': 2,
  'min_samples_split': 4,
  'min_weight_fraction_leaf': 0.0})

cboost.fit(X_train, y_train)
aboost.fit(X_train, y_train)
rf.fit(X_train, y_train)


Learning rate set to 0.04582
0:	learn: 0.6147733	total: 10.6ms	remaining: 10.6s
1:	learn: 0.5563934	total: 24ms	remaining: 12s
2:	learn: 0.5079931	total: 38.8ms	remaining: 12.9s
3:	learn: 0.4628135	total: 49.4ms	remaining: 12.3s
4:	learn: 0.4257306	total: 59.6ms	remaining: 11.9s
5:	learn: 0.3951901	total: 70ms	remaining: 11.6s
6:	learn: 0.3676773	total: 80ms	remaining: 11.3s
7:	learn: 0.3429853	total: 90.5ms	remaining: 11.2s
8:	learn: 0.3256125	total: 100ms	remaining: 11.1s
9:	learn: 0.3075893	total: 111ms	remaining: 11s
10:	learn: 0.2963993	total: 121ms	remaining: 10.9s
11:	learn: 0.2847436	total: 131ms	remaining: 10.8s
12:	learn: 0.2715039	total: 141ms	remaining: 10.7s
13:	learn: 0.2628584	total: 151ms	remaining: 10.7s
14:	learn: 0.2554010	total: 162ms	remaining: 10.6s
15:	learn: 0.2476189	total: 172ms	remaining: 10.6s
16:	learn: 0.2410197	total: 182ms	remaining: 10.5s
17:	learn: 0.2367964	total: 193ms	remaining: 10.5s
18:	learn: 0.2309972	total: 204ms	remaining: 10.5s
19:	learn: 0.2

168:	learn: 0.1662151	total: 2.02s	remaining: 9.96s
169:	learn: 0.1661419	total: 2.04s	remaining: 9.94s
170:	learn: 0.1660588	total: 2.05s	remaining: 9.93s
171:	learn: 0.1660029	total: 2.06s	remaining: 9.91s
172:	learn: 0.1659446	total: 2.07s	remaining: 9.89s
173:	learn: 0.1658672	total: 2.09s	remaining: 9.91s
174:	learn: 0.1657758	total: 2.1s	remaining: 9.9s
175:	learn: 0.1657100	total: 2.11s	remaining: 9.88s
176:	learn: 0.1656619	total: 2.12s	remaining: 9.86s
177:	learn: 0.1655624	total: 2.13s	remaining: 9.84s
178:	learn: 0.1655050	total: 2.14s	remaining: 9.82s
179:	learn: 0.1654433	total: 2.15s	remaining: 9.8s
180:	learn: 0.1653998	total: 2.16s	remaining: 9.79s
181:	learn: 0.1653310	total: 2.17s	remaining: 9.78s
182:	learn: 0.1652451	total: 2.19s	remaining: 9.76s
183:	learn: 0.1652058	total: 2.2s	remaining: 9.74s
184:	learn: 0.1651530	total: 2.21s	remaining: 9.72s
185:	learn: 0.1650862	total: 2.22s	remaining: 9.71s
186:	learn: 0.1649930	total: 2.23s	remaining: 9.7s
187:	learn: 0.164

344:	learn: 0.1544207	total: 3.98s	remaining: 7.56s
345:	learn: 0.1543429	total: 3.99s	remaining: 7.55s
346:	learn: 0.1542825	total: 4s	remaining: 7.53s
347:	learn: 0.1542303	total: 4.01s	remaining: 7.52s
348:	learn: 0.1541887	total: 4.02s	remaining: 7.5s
349:	learn: 0.1541734	total: 4.03s	remaining: 7.49s
350:	learn: 0.1541304	total: 4.04s	remaining: 7.47s
351:	learn: 0.1540671	total: 4.05s	remaining: 7.46s
352:	learn: 0.1540208	total: 4.06s	remaining: 7.44s
353:	learn: 0.1539704	total: 4.07s	remaining: 7.43s
354:	learn: 0.1539061	total: 4.08s	remaining: 7.42s
355:	learn: 0.1538281	total: 4.09s	remaining: 7.41s
356:	learn: 0.1538177	total: 4.1s	remaining: 7.39s
357:	learn: 0.1537710	total: 4.11s	remaining: 7.38s
358:	learn: 0.1537129	total: 4.13s	remaining: 7.37s
359:	learn: 0.1536618	total: 4.14s	remaining: 7.36s
360:	learn: 0.1536219	total: 4.16s	remaining: 7.36s
361:	learn: 0.1535911	total: 4.17s	remaining: 7.35s
362:	learn: 0.1535566	total: 4.18s	remaining: 7.33s
363:	learn: 0.153

512:	learn: 0.1461777	total: 5.95s	remaining: 5.65s
513:	learn: 0.1461395	total: 5.96s	remaining: 5.63s
514:	learn: 0.1461012	total: 5.97s	remaining: 5.62s
515:	learn: 0.1460634	total: 5.98s	remaining: 5.61s
516:	learn: 0.1460046	total: 5.99s	remaining: 5.6s
517:	learn: 0.1459940	total: 6s	remaining: 5.58s
518:	learn: 0.1459640	total: 6.01s	remaining: 5.57s
519:	learn: 0.1458963	total: 6.03s	remaining: 5.56s
520:	learn: 0.1458786	total: 6.03s	remaining: 5.55s
521:	learn: 0.1458243	total: 6.04s	remaining: 5.54s
522:	learn: 0.1457592	total: 6.05s	remaining: 5.52s
523:	learn: 0.1457158	total: 6.07s	remaining: 5.51s
524:	learn: 0.1456523	total: 6.08s	remaining: 5.5s
525:	learn: 0.1456063	total: 6.08s	remaining: 5.48s
526:	learn: 0.1455445	total: 6.09s	remaining: 5.47s
527:	learn: 0.1454974	total: 6.11s	remaining: 5.46s
528:	learn: 0.1454528	total: 6.12s	remaining: 5.44s
529:	learn: 0.1454276	total: 6.14s	remaining: 5.44s
530:	learn: 0.1453700	total: 6.15s	remaining: 5.43s
531:	learn: 0.145

682:	learn: 0.1387582	total: 7.86s	remaining: 3.65s
683:	learn: 0.1387155	total: 7.87s	remaining: 3.64s
684:	learn: 0.1386504	total: 7.88s	remaining: 3.62s
685:	learn: 0.1386419	total: 7.89s	remaining: 3.61s
686:	learn: 0.1385914	total: 7.91s	remaining: 3.6s
687:	learn: 0.1385593	total: 7.92s	remaining: 3.59s
688:	learn: 0.1385237	total: 7.93s	remaining: 3.58s
689:	learn: 0.1385187	total: 7.94s	remaining: 3.56s
690:	learn: 0.1384699	total: 7.95s	remaining: 3.55s
691:	learn: 0.1384311	total: 7.96s	remaining: 3.54s
692:	learn: 0.1383984	total: 7.97s	remaining: 3.53s
693:	learn: 0.1383365	total: 7.98s	remaining: 3.52s
694:	learn: 0.1382948	total: 7.99s	remaining: 3.51s
695:	learn: 0.1382300	total: 8s	remaining: 3.49s
696:	learn: 0.1381950	total: 8.01s	remaining: 3.48s
697:	learn: 0.1381605	total: 8.02s	remaining: 3.47s
698:	learn: 0.1381580	total: 8.03s	remaining: 3.46s
699:	learn: 0.1381280	total: 8.04s	remaining: 3.45s
700:	learn: 0.1380964	total: 8.06s	remaining: 3.44s
701:	learn: 0.13

850:	learn: 0.1316681	total: 9.83s	remaining: 1.72s
851:	learn: 0.1316127	total: 9.84s	remaining: 1.71s
852:	learn: 0.1315718	total: 9.85s	remaining: 1.7s
853:	learn: 0.1315491	total: 9.86s	remaining: 1.69s
854:	learn: 0.1315129	total: 9.87s	remaining: 1.67s
855:	learn: 0.1314566	total: 9.88s	remaining: 1.66s
856:	learn: 0.1314258	total: 9.9s	remaining: 1.65s
857:	learn: 0.1313769	total: 9.91s	remaining: 1.64s
858:	learn: 0.1313292	total: 9.92s	remaining: 1.63s
859:	learn: 0.1312834	total: 9.93s	remaining: 1.61s
860:	learn: 0.1312576	total: 9.94s	remaining: 1.6s
861:	learn: 0.1312185	total: 9.95s	remaining: 1.59s
862:	learn: 0.1311682	total: 9.96s	remaining: 1.58s
863:	learn: 0.1311287	total: 9.97s	remaining: 1.57s
864:	learn: 0.1310793	total: 9.98s	remaining: 1.56s
865:	learn: 0.1310053	total: 9.99s	remaining: 1.54s
866:	learn: 0.1309654	total: 10s	remaining: 1.53s
867:	learn: 0.1309340	total: 10s	remaining: 1.52s
868:	learn: 0.1308939	total: 10s	remaining: 1.51s
869:	learn: 0.1308455

RandomForestClassifier(min_samples_leaf=2, min_samples_split=4)

In [37]:
cboost_y = cboost.predict(X_test)
aboost_y = aboost.predict(X_test)
rf_y = rf.predict(X_test)

f_scores_test = [f1_score(y_test, y_pred) for y_pred in [cboost_y, aboost_y, rf_y]]

In [38]:
comparison = pd.DataFrame({'CV Train score': best_score_test,
                         'Test score': f_scores_test},
                         index = ['CatBoost', 'AdaBoost', 'RandomForest'])

In [39]:
comparison

,CV Train score,Test score
CatBoost,0.589679,0.597742
AdaBoost,0.500584,0.487233
RandomForest,0.557835,0.542222


Главный плюс катбуста - умение работать с категориальными признаками, поэтому попробуем обучить с учетом этого. А еще можно заменить nanы строками

In [105]:
df1 = df.copy()
df1.fillna('NaN', inplace = True)
X_train1, X_test1, y_train1, y_test1 = train_test_split(df, df['y'], test_size = 0.2, random_state = 69)

In [115]:
boosting_model = CatBoostClassifier(n_estimators=200, silent=True,
                                             cat_features=cat_cols,
                                             eval_metric='AUC')
grid_cat = boosting_model.grid_search({'l2_leaf_reg': np.linspace(0, 1, 10)}, 
                           X_train1.drop('y', axis = 1).astype(str), 
                           y_train1, plot=True, refit=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 0.9474311271
bestIteration = 195

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	loss: 0.9474311	best: 0.9474311 (0)	total: 12.7s	remaining: 1m 53s

bestTest = 0.9477493033
bestIteration = 199

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
1:	loss: 0.9477493	best: 0.9477493 (1)	total: 27.8s	remaining: 1m 51s

bestTest = 0.9478166314
bestIteration = 198

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
2:	loss: 0.9478166	best: 0.9478166 (2)	total: 44.8s	remaining: 1m 44s

bestTest = 0.9473646997
bestIteration = 199

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
3:	loss: 0.9473647	best: 0.9478166 (2)	total: 1m 1s	remaining: 1m 31s


In [117]:
grid_cat['params']

{'l2_leaf_reg': 0.8888888888888888}

In [123]:
cat_best = CatBoostClassifier(n_estimators=200, silent=True, cat_features=cat_cols, l2_leaf_reg = 0.88)
cat_best.fit(X_train1.drop('y', axis = 1).astype(str), y_train1)
y_pred1 = cat_best.predict(X_test1.drop('y', axis = 1).astype(str))
best_cat_f1 = f1_score(y_test1, y_pred1)
best_cat_f1

0.583743842364532

Получилось хуже))). Возможно из-за пропусков